# AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions

## Giới thiệu Paper

- **Tên paper:** AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions
- **Tác giả:** Ziming Li, Qianbo Zang, David Ma, Jiawei Guo, Tuney Zheng, Minghao Liu, Xinyao Niu, Yue Wang, Jian Yang, Jiaheng Liu, Wanjun Zhong, Wangchunshu Zhou, Wenhao Huang, Ge Zhang
- **Link:** [arXiv:2410.20424v3](https://arxiv.org/abs/2410.20424)
- **Tóm tắt:** 
  > Paper giới thiệu AutoKaggle - một framework đa tác tử giúp tự động hóa quy trình giải quyết các bài toán khoa học dữ liệu trong các cuộc thi Kaggle. AutoKaggle triển khai quy trình phát triển lặp kết hợp thực thi code, gỡ lỗi và kiểm thử đơn vị toàn diện để đảm bảo tính chính xác của code. Framework này cung cấp luồng công việc tùy biến cao, cho phép người dùng can thiệp ở mỗi giai đoạn, tích hợp trí tuệ tự động với chuyên môn của con người. Nền tảng của giải pháp là bộ công cụ khoa học dữ liệu đã được xác thực, bao gồm các hàm dùng cho làm sạch dữ liệu, kỹ thuật đặc trưng và mô hình hóa. Đánh giá trên 8 cuộc thi Kaggle cho thấy AutoKaggle đạt tỷ lệ nộp bài hợp lệ 0.85 và điểm toàn diện 0.82, chứng minh hiệu quả và tính thực tiễn trong xử lý các tác vụ khoa học dữ liệu phức tạp.

## Cài đặt môi trường và thư viện

In [ ]:
!pip install -q langchain langchain-openai langchain-community langchain-core
!pip install -q pandas numpy matplotlib seaborn scikit-learn
!pip install -q langgraph
!pip install -q langchain-anthropic
!pip install -q deepeval
!pip install -q langfuse

In [ ]:
# Import các thư viện cần thiết
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional, Tuple, Any, Union
from enum import Enum

# Import các thư viện LangChain và LangGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser

# Import LangGraph
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor

# Để đánh giá
from deepeval.metrics import FactualConsistencyMetric
from deepeval.test_case import LLMTestCase

# Thiết lập môi trường
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY_HERE"
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

## 1. Cấu trúc tổng quát của AutoKaggle

AutoKaggle là một framework đa tác tử dựa trên phase, được thiết kế để giải quyết các cuộc thi khoa học dữ liệu trên Kaggle. Framework này bao gồm hai thành phần chính:

1. **Luồng công việc dựa trên phase (Phase-based Workflow)**: Chia quy trình khoa học dữ liệu thành 6 giai đoạn chính:
   - Hiểu biết nền tảng (Background Understanding)
   - Phân tích dữ liệu khám phá sơ bộ (Preliminary EDA)
   - Làm sạch dữ liệu (Data Cleaning)
   - Phân tích dữ liệu khám phá chuyên sâu (In-depth EDA)
   - Kỹ thuật đặc trưng (Feature Engineering)
   - Xây dựng, xác thực và dự đoán mô hình (Model Building, Validation and Prediction)

2. **Hệ thống đa tác tử (Multi-agent System)**: Bao gồm 5 tác tử chuyên biệt:
   - Reader: Đọc và tóm tắt thông tin
   - Planner: Lập kế hoạch cho các nhiệm vụ
   - Developer: Viết và thực thi code
   - Reviewer: Đánh giá kết quả
   - Summarizer: Tạo báo cáo tổng hợp

Dưới đây, chúng ta sẽ triển khai một phiên bản đơn giản hóa của AutoKaggle sử dụng LangChain và LangGraph.

## 2. Định nghĩa các Phase và State

In [ ]:
# Định nghĩa các phase trong AutoKaggle
class Phase(Enum):
    BACKGROUND_UNDERSTANDING = "Background Understanding"
    PRELIMINARY_EDA = "Preliminary EDA"
    DATA_CLEANING = "Data Cleaning"
    INDEPTH_EDA = "In-depth EDA"
    FEATURE_ENGINEERING = "Feature Engineering"
    MODEL_BUILDING = "Model Building, Validation and Prediction"
    COMPLETED = "Completed"

# State của hệ thống
class AutoKaggleState:
    def __init__(self):
        self.current_phase = Phase.BACKGROUND_UNDERSTANDING
        self.competition_info = {}
        self.data = {}
        self.clean_data = {}
        self.feature_data = {}
        self.model_results = {}
        self.errors = []
        self.reports = {}
        self.plans = {}
        self.code = {}
        self.execution_results = {}
        self.test_results = {}
    
    def update_phase(self, phase: Phase):
        self.current_phase = phase
    
    def get_current_phase(self):
        return self.current_phase
    
    def add_report(self, phase: Phase, report: str):
        self.reports[phase.value] = report
    
    def add_plan(self, phase: Phase, plan: str):
        self.plans[phase.value] = plan
    
    def add_code(self, phase: Phase, code: str):
        self.code[phase.value] = code
    
    def add_execution_result(self, phase: Phase, result: str):
        self.execution_results[phase.value] = result
    
    def add_test_result(self, phase: Phase, result: Dict):
        self.test_results[phase.value] = result
    
    def add_error(self, phase: Phase, error: str):
        if not phase.value in self.errors:
            self.errors.append({"phase": phase.value, "error": error})
    
    def to_dict(self):
        return {
            "current_phase": self.current_phase.value,
            "competition_info": self.competition_info,
            "reports": self.reports,
            "plans": self.plans,
            "code": self.code,
            "execution_results": self.execution_results,
            "test_results": self.test_results,
            "errors": self.errors
        }

## 3. Triển khai các tác tử (Agents)

Trong triển khai này, chúng ta sẽ tập trung vào 5 tác tử chính của AutoKaggle:

In [ ]:
# 1. Reader Agent - Đọc và tóm tắt thông tin
class ReaderAgent:
    def __init__(self, model: BaseChatModel):
        self.model = model
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a Reader agent specialized in analyzing competition information.
            Your task is to read and summarize the competition overview, understand the data files and structure.
            Provide a comprehensive analysis covering:
            1. Competition Overview
            2. Files and their purpose
            3. Problem Definition
            4. Data Information including data types and descriptions
            5. Target Variable
            6. Evaluation Metrics
            7. Submission Format
            8. Other key aspects"""),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, input_data: Dict) -> str:
        result = self.chain.invoke({"input": input_data.get("content", ""), "history": []})
        return result

# 2. Planner Agent - Tạo kế hoạch cho các nhiệm vụ
class PlannerAgent:
    def __init__(self, model: BaseChatModel):
        self.model = model
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a Planner agent responsible for creating task plans.
            Your job is to analyze the current phase and create a detailed plan with the following structure:
            1. Break down the phase into a maximum of 4 key tasks
            2. For each task, provide:
               - Clear objective
               - Required methods and approaches
               - Expected outputs
               - Constraints and considerations
            Focus only on the current phase and be specific about each feature to process."""),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, input_data: Dict) -> str:
        phase = input_data.get("phase", "")
        state_info = input_data.get("state_info", "")
        result = self.chain.invoke({
            "input": f"Current phase: {phase}\n\nState information:\n{state_info}\n\nCreate a detailed plan for this phase.",
            "history": []
        })
        return result

# 3. Developer Agent - Viết và thực thi code
class DeveloperAgent:
    def __init__(self, model: BaseChatModel):
        self.model = model
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a Developer agent specialized in implementing code according to plans.
            Your task is to write code for data science tasks based on the provided plan.
            You should:
            1. Create well-structured, efficient, and correct Python code
            2. Follow the plan strictly and implement all specified tasks
            3. Include detailed comments to explain complex operations
            4. Generate code that can be directly executed
            5. Prepare for error handling and validation
            
            When there are errors in the code, you should debug it by:
            1. Analyzing the error message
            2. Identifying the root cause
            3. Correcting the code
            4. Retesting to confirm the fix"""),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, input_data: Dict) -> str:
        plan = input_data.get("plan", "")
        phase = input_data.get("phase", "")
        error_message = input_data.get("error_message", "")
        previous_code = input_data.get("previous_code", "")
        
        if error_message:
            task_description = f"Debug the following code for phase '{phase}':\n\n{previous_code}\n\nError message:\n{error_message}"
        else:
            task_description = f"Implement the code for phase '{phase}' based on the following plan:\n\n{plan}"
        
        result = self.chain.invoke({"input": task_description, "history": []})
        return result

# 4. Reviewer Agent - Đánh giá kết quả
class ReviewerAgent:
    def __init__(self, model: BaseChatModel):
        self.model = model
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a Reviewer agent responsible for evaluating the quality and correctness of code and results.
            Your task is to review the code and execution results to ensure they meet the requirements and objectives.
            Evaluate based on:
            1. Code correctness and efficiency
            2. Alignment with the original plan
            3. Quality of the outputs and results
            4. Compliance with best practices
            Provide detailed feedback highlighting strengths and areas for improvement."""),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, input_data: Dict) -> str:
        code = input_data.get("code", "")
        plan = input_data.get("plan", "")
        execution_result = input_data.get("execution_result", "")
        phase = input_data.get("phase", "")
        
        review_task = f"Review the following code for phase '{phase}':\n\n{code}\n\nOriginal plan:\n{plan}\n\nExecution results:\n{execution_result}"
        result = self.chain.invoke({"input": review_task, "history": []})
        return result

# 5. Summarizer Agent - Tạo báo cáo tổng hợp
class SummarizerAgent:
    def __init__(self, model: BaseChatModel):
        self.model = model
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a Summarizer agent specialized in creating comprehensive reports.
            Your task is to summarize the work completed in a phase, analyzing the following:
            1. Key findings and insights
            2. Actions taken and their rationale
            3. Impact on the overall project
            4. Challenges faced and how they were addressed
            5. Recommendations for subsequent phases
            Create a clear, well-structured report that highlights the most important aspects and lessons learned."""),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, input_data: Dict) -> str:
        phase = input_data.get("phase", "")
        plan = input_data.get("plan", "")
        code = input_data.get("code", "")
        execution_result = input_data.get("execution_result", "")
        review = input_data.get("review", "")
        
        summary_task = f"Create a comprehensive summary report for phase '{phase}':\n\nPlan:\n{plan}\n\nCode implementation:\n{code}\n\nExecution results:\n{execution_result}\n\nReview:\n{review}"
        result = self.chain.invoke({"input": summary_task, "history": []})
        return result

## 4. Triển khai các công cụ Machine Learning

Thư viện công cụ máy học trong AutoKaggle được phân loại thành ba bộ công cụ chính: làm sạch dữ liệu (data cleaning), kỹ thuật đặc trưng (feature engineering) và xây dựng mô hình, xác thực và dự đoán (model building, validation, and prediction). Dưới đây là một số công cụ quan trọng được triển khai:

In [ ]:
# 1. Data Cleaning Tools
class DataCleaningTools:
    @staticmethod
    def fill_missing_values(df: pd.DataFrame, strategy: Dict[str, str]) -> pd.DataFrame:
        """
        Fill missing values in a dataframe based on specified strategies.
        
        Parameters:
        - df: The input dataframe
        - strategy: Dictionary with column names as keys and filling strategies as values
                   (e.g., {"col1": "mean", "col2": "median", "col3": "mode", "col4": "0"})
        
        Returns:
        - DataFrame with filled missing values
        """
        df_copy = df.copy()
        
        for column, method in strategy.items():
            if column not in df_copy.columns:
                continue
                
            if pd.api.types.is_numeric_dtype(df_copy[column]):
                if method == "mean":
                    df_copy[column].fillna(df_copy[column].mean(), inplace=True)
                elif method == "median":
                    df_copy[column].fillna(df_copy[column].median(), inplace=True)
                elif method == "mode":
                    df_copy[column].fillna(df_copy[column].mode()[0], inplace=True)
                elif method == "0":
                    df_copy[column].fillna(0, inplace=True)
                elif method == "none":
                    pass
                else:
                    try:
                        df_copy[column].fillna(float(method), inplace=True)
                    except ValueError:
                        pass
            else:
                if method == "mode":
                    df_copy[column].fillna(df_copy[column].mode()[0] if not df_copy[column].mode().empty else "", inplace=True)
                elif method == "unknown":
                    df_copy[column].fillna("Unknown", inplace=True)
                elif method == "none":
                    pass
                else:
                    df_copy[column].fillna(method, inplace=True)
                    
        return df_copy
    
    @staticmethod
    def remove_duplicates(df: pd.DataFrame, subset: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Remove duplicate rows from a dataframe.
        
        Parameters:
        - df: The input dataframe
        - subset: List of column names to consider for identifying duplicates (optional)
        
        Returns:
        - DataFrame with duplicates removed
        """
        return df.drop_duplicates(subset=subset, keep='first')
    
    @staticmethod
    def detect_and_handle_outliers_zscore(df: pd.DataFrame, columns: List[str], threshold: float = 3.0, 
                                         strategy: str = "clip") -> pd.DataFrame:
        """
        Detect and handle outliers using Z-score method.
        
        Parameters:
        - df: The input dataframe
        - columns: List of columns to check for outliers
        - threshold: Z-score threshold (default: 3.0)
        - strategy: Handling strategy - "clip", "remove", or "replace" (default: "clip")
        
        Returns:
        - DataFrame with outliers handled
        """
        df_copy = df.copy()
        
        for column in columns:
            if column not in df_copy.columns or not pd.api.types.is_numeric_dtype(df_copy[column]):
                continue
                
            # Compute z-scores
            z_scores = np.abs((df_copy[column] - df_copy[column].mean()) / df_copy[column].std())
            outliers = z_scores > threshold
            
            if strategy == "clip":
                lower_bound = df_copy[column].mean() - threshold * df_copy[column].std()
                upper_bound = df_copy[column].mean() + threshold * df_copy[column].std()
                df_copy.loc[z_scores > threshold, column] = np.clip(df_copy.loc[z_scores > threshold, column], 
                                                                  lower_bound, upper_bound)
            elif strategy == "remove":
                df_copy = df_copy[~outliers]
            elif strategy == "replace":
                df_copy.loc[outliers, column] = df_copy[column].mean()
                
        return df_copy
    
    @staticmethod
    def convert_data_types(df: pd.DataFrame, type_mappings: Dict[str, str]) -> pd.DataFrame:
        """
        Convert data types of specified columns.
        
        Parameters:
        - df: The input dataframe
        - type_mappings: Dictionary with column names as keys and desired data types as values
                        (e.g., {"col1": "int", "col2": "float", "col3": "category", "col4": "datetime"})
        
        Returns:
        - DataFrame with converted data types
        """
        df_copy = df.copy()
        
        for column, dtype in type_mappings.items():
            if column not in df_copy.columns:
                continue
                
            try:
                if dtype == "int":
                    df_copy[column] = pd.to_numeric(df_copy[column], errors='coerce').fillna(0).astype(int)
                elif dtype == "float":
                    df_copy[column] = pd.to_numeric(df_copy[column], errors='coerce')
                elif dtype == "category":
                    df_copy[column] = df_copy[column].astype('category')
                elif dtype == "datetime":
                    df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
                elif dtype == "string" or dtype == "str":
                    df_copy[column] = df_copy[column].astype(str)
                else:
                    df_copy[column] = df_copy[column].astype(dtype)
            except Exception as e:
                print(f"Error converting {column} to {dtype}: {e}")
                
        return df_copy

# 2. Feature Engineering Tools
class FeatureEngineeringTools:
    @staticmethod
    def one_hot_encode(df: pd.DataFrame, columns: List[str], drop_original: bool = True) -> pd.DataFrame:
        """
        One-hot encode categorical columns.
        
        Parameters:
        - df: The input dataframe
        - columns: List of categorical columns to encode
        - drop_original: Whether to drop the original columns (default: True)
        
        Returns:
        - DataFrame with one-hot encoded features
        """
        df_copy = df.copy()
        
        for column in columns:
            if column not in df_copy.columns:
                continue
                
            dummies = pd.get_dummies(df_copy[column], prefix=column, dummy_na=False)
            df_copy = pd.concat([df_copy, dummies], axis=1)
            
            if drop_original:
                df_copy.drop(column, axis=1, inplace=True)
                
        return df_copy
    
    @staticmethod
    def frequency_encode(df: pd.DataFrame, columns: List[str], drop_original: bool = True) -> pd.DataFrame:
        """
        Frequency encode categorical columns.
        
        Parameters:
        - df: The input dataframe
        - columns: List of categorical columns to encode
        - drop_original: Whether to drop the original columns (default: True)
        
        Returns:
        - DataFrame with frequency encoded features
        """
        df_copy = df.copy()
        
        for column in columns:
            if column not in df_copy.columns:
                continue
                
            frequency = df_copy[column].value_counts(normalize=True).to_dict()
            df_copy[f"{column}_freq"] = df_copy[column].map(frequency)
            
            if drop_original:
                df_copy.drop(column, axis=1, inplace=True)
                
        return df_copy
    
    @staticmethod
    def correlation_feature_selection(df: pd.DataFrame, target_column: str, threshold: float = 0.05) -> List[str]:
        """
        Select features based on correlation with target.
        
        Parameters:
        - df: The input dataframe
        - target_column: The target variable column name
        - threshold: Minimum absolute correlation threshold (default: 0.05)
        
        Returns:
        - List of selected feature names
        """
        if target_column not in df.columns:
            return []
            
        numeric_df = df.select_dtypes(include=['number'])
        
        if target_column not in numeric_df.columns:
            return []
            
        correlations = numeric_df.corr()[target_column].abs()
        selected_features = correlations[correlations > threshold].index.tolist()
        
        # Remove target column from the list
        if target_column in selected_features:
            selected_features.remove(target_column)
            
        return selected_features
    
    @staticmethod
    def scale_features(df: pd.DataFrame, columns: List[str], method: str = "standard") -> pd.DataFrame:
        """
        Scale numeric features.
        
        Parameters:
        - df: The input dataframe
        - columns: List of numeric columns to scale
        - method: Scaling method - "standard" or "minmax" (default: "standard")
        
        Returns:
        - DataFrame with scaled features
        """
        from sklearn.preprocessing import StandardScaler, MinMaxScaler
        
        df_copy = df.copy()
        features_to_scale = [col for col in columns if col in df_copy.columns]
        
        if not features_to_scale:
            return df_copy
            
        if method == "standard":
            scaler = StandardScaler()
        elif method == "minmax":
            scaler = MinMaxScaler()
        else:
            raise ValueError("Method must be 'standard' or 'minmax'")
            
        df_copy[features_to_scale] = scaler.fit_transform(df_copy[features_to_scale])
        return df_copy

# 3. Model Building, Validation, and Prediction Tools
class ModelTools:
    @staticmethod
    def train_and_validate_and_select_best_model(X_train, y_train, X_val, y_val, 
                                                models_config: List[Dict], 
                                                problem_type: str,
                                                evaluation_metric: str) -> Tuple[Any, Dict, pd.DataFrame]:
        """
        Train multiple models, validate them, and select the best one.
        
        Parameters:
        - X_train: Training features
        - y_train: Training target
        - X_val: Validation features
        - y_val: Validation target
        - models_config: List of model configurations with 'model_type' and 'params'
        - problem_type: 'classification' or 'regression'
        - evaluation_metric: Metric to evaluate models (e.g., 'accuracy', 'f1', 'rmse')
        
        Returns:
        - Tuple of (best_model, best_model_config, results_df)
        """
        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                                  mean_squared_error, mean_absolute_error, r2_score
        from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
        from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
        from sklearn.svm import SVC, SVR
        from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
        import xgboost as xgb
        import lightgbm as lgb
        
        results = []
        models = []
        
        # Define metric function
        if problem_type == "classification":
            if evaluation_metric == "accuracy":
                metric_func = accuracy_score
            elif evaluation_metric == "f1":
                metric_func = f1_score
            elif evaluation_metric == "precision":
                metric_func = precision_score
            elif evaluation_metric == "recall":
                metric_func = recall_score
            else:
                metric_func = accuracy_score
        else:  # regression
            if evaluation_metric == "rmse":
                metric_func = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))
            elif evaluation_metric == "mae":
                metric_func = mean_absolute_error
            elif evaluation_metric == "r2":
                metric_func = r2_score
            else:
                metric_func = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))
        
        for config in models_config:
            model_type = config['model_type']
            params = config.get('params', {})
            
            try:
                # Initialize the model based on problem type
                if problem_type == "classification":
                    if model_type == "RandomForest":
                        model = RandomForestClassifier(**params)
                    elif model_type == "GradientBoosting":
                        model = GradientBoostingClassifier(**params)
                    elif model_type == "LogisticRegression":
                        model = LogisticRegression(**params)
                    elif model_type == "SVM":
                        model = SVC(**params, probability=True)
                    elif model_type == "KNN":
                        model = KNeighborsClassifier(**params)
                    elif model_type == "XGBoost":
                        model = xgb.XGBClassifier(**params)
                    elif model_type == "LightGBM":
                        model = lgb.LGBMClassifier(**params)
                    else:
                        continue
                else:  # regression
                    if model_type == "RandomForest":
                        model = RandomForestRegressor(**params)
                    elif model_type == "GradientBoosting":
                        model = GradientBoostingRegressor(**params)
                    elif model_type == "LinearRegression":
                        model = LinearRegression(**params)
                    elif model_type == "Ridge":
                        model = Ridge(**params)
                    elif model_type == "Lasso":
                        model = Lasso(**params)
                    elif model_type == "SVM":
                        model = SVR(**params)
                    elif model_type == "KNN":
                        model = KNeighborsRegressor(**params)
                    elif model_type == "XGBoost":
                        model = xgb.XGBRegressor(**params)
                    elif model_type == "LightGBM":
                        model = lgb.LGBMRegressor(**params)
                    else:
                        continue
                        
                # Train the model
                model.fit(X_train, y_train)
                models.append(model)
                
                # Make predictions
                train_preds = model.predict(X_train)
                val_preds = model.predict(X_val)
                
                # Calculate metrics
                train_score = metric_func(y_train, train_preds)
                val_score = metric_func(y_val, val_preds)
                
                # Store results
                results.append({
                    "model_type": model_type,
                    "params": params,
                    "train_score": train_score,
                    "val_score": val_score
                })
                
            except Exception as e:
                print(f"Error training {model_type}: {e}")
                
        # Create results dataframe
        results_df = pd.DataFrame(results)
        
        if results_df.empty:
            return None, {}, pd.DataFrame()
        
        # Select best model based on validation score
        if problem_type == "regression" and evaluation_metric in ["rmse", "mae"]:
            # Lower is better for these metrics
            best_idx = results_df["val_score"].idxmin()
        else:
            # Higher is better for other metrics
            best_idx = results_df["val_score"].idxmax()
            
        best_model = models[best_idx]
        best_config = results[best_idx]
        
        return best_model, best_config, results_df

## 5. Triển khai gỡ lỗi và kiểm thử lặp lại

AutoKaggle sử dụng phương pháp gỡ lỗi và kiểm thử lặp lại để đảm bảo code được tạo ra đáp ứng các yêu cầu và đáng tin cậy.

In [ ]:
class IterativeDebugger:
    def __init__(self, developer: DeveloperAgent, max_attempts: int = 5):
        self.developer = developer
        self.max_attempts = max_attempts
        self.error_history = []
    
    def execute_code(self, code: str) -> Tuple[bool, str]:
        """
        Execute code and capture errors.
        
        Parameters:
        - code: Python code to execute
        
        Returns:
        - Tuple of (success flag, output/error message)
        """
        try:
            # In a real implementation, we would use tools to execute code safely
            # For demonstration, we'll simulate this behavior
            result = "[Simulation] Code executed successfully"
            return True, result
        except Exception as e:
            return False, str(e)
    
    def run_unit_tests(self, code: str, test_specs: List[Dict]) -> Tuple[bool, Dict]:
        """
        Run unit tests on code.
        
        Parameters:
        - code: Python code to test
        - test_specs: List of test specifications
        
        Returns:
        - Tuple of (all_passed flag, test results)
        """
        # In a real implementation, we would use unit testing frameworks
        # For demonstration, we'll simulate this behavior
        test_results = {}
        all_passed = True
        
        for test in test_specs:
            test_id = test.get("id", "unknown")
            passed = True  # Simulate all tests passing
            result = "[Simulation] Test passed"
            
            test_results[test_id] = {
                "passed": passed,
                "result": result
            }
            
            if not passed:
                all_passed = False
        
        return all_passed, test_results
    
    def debug_code(self, code: str, error_message: str, phase: str) -> str:
        """
        Debug code using the Developer agent.
        
        Parameters:
        - code: Code to debug
        - error_message: Error message from execution
        - phase: Current phase name
        
        Returns:
        - Debugged code
        """
        input_data = {
            "previous_code": code,
            "error_message": error_message,
            "phase": phase
        }
        
        debugged_code = self.developer.execute(input_data)
        # Extract code blocks if the result contains explanations
        if "```python" in debugged_code and "```" in debugged_code:
            code_blocks = []
            in_code_block = False
            for line in debugged_code.split("\n"):
                if line.strip() == "```python" or line.strip() == "```python3":
                    in_code_block = True
                    continue
                elif line.strip() == "```" and in_code_block:
                    in_code_block = False
                    continue
                elif in_code_block:
                    code_blocks.append(line)
            
            if code_blocks:
                debugged_code = "\n".join(code_blocks)
        
        return debugged_code
    
    def iterative_debug_and_test(self, code: str, test_specs: List[Dict], phase: str) -> Tuple[str, bool, str, Dict]:
        """
        Perform iterative debugging and testing.
        
        Parameters:
        - code: Initial code
        - test_specs: Unit test specifications
        - phase: Current phase name
        
        Returns:
        - Tuple of (final_code, success flag, execution_result, test_results)
        """
        current_code = code
        success = False
        execution_result = ""
        test_results = {}
        attempts = 0
        self.error_history = []
        
        while attempts < self.max_attempts and not success:
            # Execute the code
            execution_success, execution_result = self.execute_code(current_code)
            
            if execution_success:
                # Run unit tests
                tests_passed, test_results = self.run_unit_tests(current_code, test_specs)
                
                if tests_passed:
                    success = True
                    break
                else:
                    # Debug test failures
                    error_msg = "Unit tests failed: " + str(test_results)
                    self.error_history.append(error_msg)
                    current_code = self.debug_code(current_code, error_msg, phase)
            else:
                # Debug execution errors
                self.error_history.append(execution_result)
                
                # Check if similar errors are recurring
                if attempts >= 3 and len(set(self.error_history[-3:])) == 1:
                    # If the same error occurs multiple times, regenerate code from scratch
                    input_data = {"phase": phase, "plan": "Please completely rewrite the code to fix recurring errors."}
                    current_code = self.developer.execute(input_data)
                    self.error_history = []
                else:
                    current_code = self.debug_code(current_code, execution_result, phase)
            
            attempts += 1
        
        return current_code, success, execution_result, test_results

## 6. Xây dựng luồng công việc với LangGraph

Sử dụng LangGraph để thiết lập luồng làm việc phase-based của AutoKaggle.

In [ ]:
class AutoKaggle:
    def __init__(self, model_name: str = "gpt-4o"):
        # Khởi tạo LLM
        if "anthropic" in model_name.lower() or "claude" in model_name.lower():
            self.model = ChatAnthropic(model=model_name)
        else:  # Default to OpenAI
            self.model = ChatOpenAI(model=model_name)
            
        # Khởi tạo các agent
        self.reader = ReaderAgent(self.model)
        self.planner = PlannerAgent(self.model)
        self.developer = DeveloperAgent(self.model)
        self.reviewer = ReviewerAgent(self.model)
        self.summarizer = SummarizerAgent(self.model)
        
        # Khởi tạo debugger
        self.debugger = IterativeDebugger(self.developer)
        
        # Khởi tạo state
        self.state = AutoKaggleState()
        
        # Xây dựng workflow graph
        self._build_graph()
    
    def _build_graph(self):
        """
        Xây dựng đồ thị LangGraph cho workflow của AutoKaggle
        """
        self.graph = StateGraph(AutoKaggleState)
        
        # Định nghĩa các node cho các phase
        self.graph.add_node("background_understanding", self._execute_background_understanding)
        self.graph.add_node("preliminary_eda", self._execute_preliminary_eda)
        self.graph.add_node("data_cleaning", self._execute_data_cleaning)
        self.graph.add_node("indepth_eda", self._execute_indepth_eda)
        self.graph.add_node("feature_engineering", self._execute_feature_engineering)
        self.graph.add_node("model_building", self._execute_model_building)
        
        # Định nghĩa các cạnh và điều kiện chuyển tiếp
        self.graph.add_edge("background_understanding", "preliminary_eda")
        self.graph.add_edge("preliminary_eda", "data_cleaning")
        self.graph.add_edge("data_cleaning", "indepth_eda")
        self.graph.add_edge("indepth_eda", "feature_engineering")
        self.graph.add_edge("feature_engineering", "model_building")
        self.graph.add_edge("model_building", END)
        
        # Compiler đồ thị
        self.workflow = self.graph.compile()
    
    def _execute_background_understanding(self, state: AutoKaggleState):
        """
        Thực hiện phase Background Understanding
        """
        # Giả định input_data chứa nội dung của overview.txt
        input_data = {"content": "[Competition overview and data information]"}
        
        # Reader đọc và tóm tắt thông tin
        report = self.reader.execute(input_data)
        state.add_report(Phase.BACKGROUND_UNDERSTANDING, report)
        
        # Cập nhật trạng thái
        state.update_phase(Phase.PRELIMINARY_EDA)
        return state
    
    def _execute_preliminary_eda(self, state: AutoKaggleState):
        """
        Thực hiện phase Preliminary EDA
        """
        # Lấy thông tin từ phase trước
        background_report = state.reports.get(Phase.BACKGROUND_UNDERSTANDING.value, "")
        
        # Planner tạo kế hoạch cho EDA sơ bộ
        input_data = {
            "phase": Phase.PRELIMINARY_EDA.value,
            "state_info": background_report
        }
        plan = self.planner.execute(input_data)
        state.add_plan(Phase.PRELIMINARY_EDA, plan)
        
        # Developer viết code cho EDA sơ bộ
        input_data = {
            "phase": Phase.PRELIMINARY_EDA.value,
            "plan": plan
        }
        code = self.developer.execute(input_data)
        
        # Thực hiện debugging và testing
        test_specs = [{"id": "test_basic", "description": "Basic functionality test"}]
        final_code, success, execution_result, test_results = self.debugger.iterative_debug_and_test(
            code, test_specs, Phase.PRELIMINARY_EDA.value
        )
        
        # Lưu kết quả
        state.add_code(Phase.PRELIMINARY_EDA, final_code)
        state.add_execution_result(Phase.PRELIMINARY_EDA, execution_result)
        state.add_test_result(Phase.PRELIMINARY_EDA, test_results)
        
        # Reviewer đánh giá kết quả
        input_data = {
            "code": final_code,
            "plan": plan,
            "execution_result": execution_result,
            "phase": Phase.PRELIMINARY_EDA.value
        }
        review = self.reviewer.execute(input_data)
        
        # Summarizer tạo báo cáo tổng hợp
        input_data = {
            "phase": Phase.PRELIMINARY_EDA.value,
            "plan": plan,
            "code": final_code,
            "execution_result": execution_result,
            "review": review
        }
        summary = self.summarizer.execute(input_data)
        state.add_report(Phase.PRELIMINARY_EDA, summary)
        
        # Cập nhật trạng thái
        state.update_phase(Phase.DATA_CLEANING)
        return state
    
    def _execute_data_cleaning(self, state: AutoKaggleState):
        """
        Thực hiện phase Data Cleaning
        """
        # Tương tự như _execute_preliminary_eda nhưng cho giai đoạn làm sạch dữ liệu
        # Trong giai đoạn này, sẽ sử dụng các công cụ DataCleaningTools
        
        # Cập nhật trạng thái
        state.update_phase(Phase.INDEPTH_EDA)
        return state
    
    def _execute_indepth_eda(self, state: AutoKaggleState):
        """
        Thực hiện phase In-depth EDA
        """
        # Tương tự như các phase khác
        
        # Cập nhật trạng thái
        state.update_phase(Phase.FEATURE_ENGINEERING)
        return state
    
    def _execute_feature_engineering(self, state: AutoKaggleState):
        """
        Thực hiện phase Feature Engineering
        """
        # Tương tự như các phase khác, sử dụng FeatureEngineeringTools
        
        # Cập nhật trạng thái
        state.update_phase(Phase.MODEL_BUILDING)
        return state
    
    def _execute_model_building(self, state: AutoKaggleState):
        """
        Thực hiện phase Model Building, Validation, and Prediction
        """
        # Tương tự như các phase khác, sử dụng ModelTools
        
        # Cập nhật trạng thái
        state.update_phase(Phase.COMPLETED)
        return state
    
    def run(self, competition_info: Dict) -> AutoKaggleState:
        """
        Chạy toàn bộ workflow AutoKaggle
        
        Parameters:
        - competition_info: Dictionary chứa thông tin về cuộc thi và dữ liệu
        
        Returns:
        - Trạng thái cuối cùng của hệ thống
        """
        self.state.competition_info = competition_info
        final_state = self.workflow.invoke(self.state)
        return final_state

## 7. Đánh giá với DeepEval

Sử dụng DeepEval để đánh giá hiệu suất của hệ thống AutoKaggle.

In [ ]:
# Đánh giá các kết quả được tạo ra bởi AutoKaggle
class AutoKaggleEvaluator:
    @staticmethod
    def evaluate_phase_report(phase_name: str, report: str, ground_truth: str) -> float:
        """
        Đánh giá báo cáo của một phase dựa trên tính nhất quán thực tế
        
        Parameters:
        - phase_name: Tên của phase
        - report: Báo cáo cần đánh giá
        - ground_truth: Thông tin đúng để so sánh
        
        Returns:
        - Điểm đánh giá
        """
        test_case = LLMTestCase(
            input=f"Evaluate the {phase_name} phase",
            actual_output=report,
            expected_output=ground_truth
        )
        
        metric = FactualConsistencyMetric()
        metric.measure(test_case)
        return metric.score
    
    @staticmethod
    def calculate_made_submission(state: AutoKaggleState) -> bool:
        """
        Kiểm tra xem đã tạo file submission.csv chưa
        
        Parameters:
        - state: Trạng thái của hệ thống
        
        Returns:
        - True nếu đã tạo, False nếu chưa
        """
        # Giả định kiểm tra file submission.csv trong kết quả thực thi
        return "submission.csv" in str(state.execution_results.get(Phase.MODEL_BUILDING.value, ""))
    
    @staticmethod
    def calculate_valid_submission(state: AutoKaggleState, expected_format: Dict) -> bool:
        """
        Kiểm tra xem file submission có đúng định dạng không
        
        Parameters:
        - state: Trạng thái của hệ thống
        - expected_format: Định dạng mong đợi của file submission
        
        Returns:
        - True nếu đúng định dạng, False nếu không
        """
        # Giả định kiểm tra định dạng file submission.csv
        return True  # Giả lập kết quả
    
    @staticmethod
    def calculate_comprehensive_score(made_submission: bool, valid_submission: bool, phase_scores: Dict[str, float]) -> float:
        """
        Tính điểm toàn diện dựa trên công thức từ paper
        
        Parameters:
        - made_submission: Có tạo file submission không
        - valid_submission: File submission có hợp lệ không
        - phase_scores: Điểm đánh giá cho từng phase
        
        Returns:
        - Điểm toàn diện
        """
        if not made_submission:
            return 0.0
            
        vs = 1.0 if valid_submission else 0.0
        anps = sum(phase_scores.values()) / len(phase_scores) if phase_scores else 0.0
        
        return 0.5 * vs + 0.5 * anps

## 8. Ví dụ minh họa

Dưới đây là một ví dụ minh họa về cách sử dụng AutoKaggle:

In [ ]:
# Tạo ví dụ minh họa
def run_example():
    # Giả định thông tin cuộc thi
    competition_info = {
        "name": "Titanic: Machine Learning from Disaster",
        "description": "Predict survival on the Titanic",
        "data_files": {
            "train.csv": "Training data with survival information",
            "test.csv": "Test data without survival information"
        },
        "target": "Survived",
        "metric": "accuracy",
        "submission_format": {
            "columns": ["PassengerId", "Survived"],
            "file_name": "submission.csv"
        }
    }
    
    # Khởi tạo AutoKaggle
    auto_kaggle = AutoKaggle(model_name="gpt-4o")
    
    # Chạy workflow
    final_state = auto_kaggle.run(competition_info)
    
    # Đánh giá kết quả
    made_submission = AutoKaggleEvaluator.calculate_made_submission(final_state)
    valid_submission = AutoKaggleEvaluator.calculate_valid_submission(
        final_state, competition_info["submission_format"]
    )
    
    # Đánh giá các báo cáo
    phase_scores = {}
    for phase in [p.value for p in list(Phase) if p != Phase.COMPLETED]:
        report = final_state.reports.get(phase, "")
        ground_truth = f"Expected report for {phase}"  # Giả định
        score = AutoKaggleEvaluator.evaluate_phase_report(phase, report, ground_truth)
        phase_scores[phase] = score
    
    # Tính điểm toàn diện
    comprehensive_score = AutoKaggleEvaluator.calculate_comprehensive_score(
        made_submission, valid_submission, phase_scores
    )
    
    return {
        "made_submission": made_submission,
        "valid_submission": valid_submission,
        "phase_scores": phase_scores,
        "comprehensive_score": comprehensive_score
    }

# Chạy ví dụ (comment lại để không thực thi thực tế)
# results = run_example()
# print(f"Comprehensive Score: {results['comprehensive_score']:.2f}")

## 9. Ánh xạ từ bài báo đến triển khai

Dưới đây là ánh xạ giữa các khái niệm trong bài báo "AutoKaggle" và cách chúng được triển khai trong notebook này:

1. **Phase-based Workflow**
   - Trong bài báo: Quy trình được chia thành 6 giai đoạn từ hiểu biết nền tảng đến xây dựng mô hình
   - Triển khai: Enum `Phase` và các phương thức `_execute_*` trong lớp `AutoKaggle`

2. **Multi-agent System**
   - Trong bài báo: 5 tác tử chuyên biệt (Reader, Planner, Developer, Reviewer, Summarizer)
   - Triển khai: Các lớp agent tương ứng và sự hợp tác thông qua LangGraph

3. **Iterative Debugging and Testing**
   - Trong bài báo: Quy trình gỡ lỗi lặp và kiểm thử đơn vị
   - Triển khai: Lớp `IterativeDebugger` và phương thức `iterative_debug_and_test`

4. **Machine Learning Tools Library**
   - Trong bài báo: Bộ công cụ toàn diện cho làm sạch dữ liệu, kỹ thuật đặc trưng và mô hình hóa
   - Triển khai: Các lớp `DataCleaningTools`, `FeatureEngineeringTools`, và `ModelTools`

5. **Comprehensive Reporting**
   - Trong bài báo: Báo cáo chi tiết sau mỗi giai đoạn
   - Triển khai: Tác tử `SummarizerAgent` và phương thức lưu trữ báo cáo trong `AutoKaggleState`

## 10. Ứng dụng thực tế và hướng phát triển

AutoKaggle là một framework mạnh mẽ có thể được ứng dụng trong nhiều lĩnh vực ngoài các cuộc thi Kaggle:

1. **Tự động hóa quy trình khoa học dữ liệu**: Giúp tự động hóa các quy trình lặp đi lặp lại trong khoa học dữ liệu, tiết kiệm thời gian và công sức.

2. **Hỗ trợ giáo dục và đào tạo**: Cung cấp công cụ học tập cho người mới bắt đầu trong lĩnh vực khoa học dữ liệu.

3. **Nghiên cứu đa tác tử**: Mô hình hợp tác đa tác tử có thể được mở rộng cho nhiều lĩnh vực khác.

4. **Kiểm thử mô hình**: Giúp kiểm tra tính mạnh mẽ và đáng tin cậy của các mô hình ML trước khi triển khai.

Các hướng phát triển trong tương lai:

1. **Tích hợp nhiều công cụ ML hơn**: Mở rộng thư viện công cụ để hỗ trợ nhiều kỹ thuật và thuật toán hơn.

2. **Hỗ trợ nhiều loại dữ liệu hơn**: Mở rộng hỗ trợ cho dữ liệu hình ảnh, văn bản và dữ liệu dạng chuỗi thời gian.

3. **Cải thiện tính tương tác**: Thêm cơ chế phản hồi của người dùng và điều chỉnh quy trình dựa trên phản hồi.

4. **Tối ưu hóa hiệu suất**: Cải thiện tốc độ và hiệu quả của framework để xử lý các tập dữ liệu lớn hơn.